<a href="https://colab.research.google.com/github/AMB-Git-Hub/BIDS_Uebungen/blob/main/Andreas_Bucher_ETLAufgaben.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# import libraries
import sqlite3
from sqlite3 import Error

In [6]:
# mount drive to access data
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
material_path = "/content/drive/MyDrive/allergy/"
!ls {material_path} #Liste aller CSV Dateien
src_db = f"/content/drive/MyDrive/source_allergy_test.db"
!ls {src_db} #Liste aller CSV Dateien

 allergy_test.db	 encounters.gsheet	    patients.csv
'careplans (1).gsheet'	 imaging_studies.csv	    payers.csv
 careplans.csv		 imaging_studies.gsheet     payer_transitions.csv
 careplans.gsheet	 immunizations.csv	    procedures.csv
 conditions.csv		 medications.csv	    procedures.gsheet
 devices.csv		'observations (1).gsheet'   providers.csv
 disease.csv		 observations.csv	    source_allergy_test.db
 disease.gsheet		 observations.gsheet	    supplies.csv
 encounters.csv		 organizations.csv
/content/drive/MyDrive/source_allergy_test.db


# Python SQLite Guide


In [9]:
# SQLite connection
def create_connection_memory():
    """ create a database connection to a database that resides
        in the memory
    """
    conn = None;
    try:
        conn = sqlite3.connect(':memory:')
        return conn
        print(sqlite3.version)
    except Error as e:
        print("Error while connecting to sqlite", e)
    # finally:
    #     if conn:
    #         conn.close()

In [10]:
# create a temporary empty database in the memory.
conn = create_connection_memory()
cur = conn.cursor()
print("Successfully Connected to SQLite")


Successfully Connected to SQLite


In [ ]:
# Create SQLite table
create_table_query = '''CREATE TABLE SqliteDb_developers (
                                id INTEGER PRIMARY KEY,
                                name TEXT NOT NULL,
                                email text NOT NULL UNIQUE,
                                joining_date datetime,
                                salary REAL NOT NULL);'''

cur.execute(create_table_query)
conn.commit()
print("SQLite table created")

SQLite table created


In [ ]:
# list of tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('SqliteDb_developers',)]


In [ ]:
# insert into table
cur.execute("INSERT INTO SqliteDb_developers (id,  name, email, salary) VALUES (1, 'name1', 'name1@gmail.com', 1000)")


In [ ]:
# insert into table-Larger example
rows = [(2, 'name2', 'name2@gmail.com', '2006-02-28', 2000),
        (3, 'name3', 'name3@gmail.com', '2006-03-05', 3000),
        (4, 'name4', 'name4@gmail.com', '2006-04-06', 4000)]
cur.executemany('insert into SqliteDb_developers values (?,?,?,?,?)', rows)
conn.commit()


In [ ]:
select_query = "SELECT * FROM SqliteDb_developers"
cur.execute(select_query)
records = cur.fetchall()
print("Total rows in table: ", len(records))
for row in records:
  print(row)

Total rows in table:  4
(1, 'name1', 'name1@gmail.com', None, 1000.0)
(2, 'name2', 'name2@gmail.com', '2006-02-28', 2000.0)
(3, 'name3', 'name3@gmail.com', '2006-03-05', 3000.0)
(4, 'name4', 'name4@gmail.com', '2006-04-06', 4000.0)


In [ ]:
# select with parameters
select_query = """select * from SqliteDb_developers where id = ?"""
cur.execute(select_query, (1,))
records = cur.fetchall()
print("Printing ID ", id)
for row in records:
    print("Name = ", row[1])
    print("Email  = ", row[2])
    print("JoiningDate  = ", row[3])
    print("Salary  = ", row[4])

Printing ID  <built-in function id>
Name =  name1
Email  =  name1@gmail.com
JoiningDate  =  None
Salary  =  1000.0


In [ ]:
# update table
cur.execute("UPDATE SqliteDb_developers SET Email=? WHERE id =?", ('Uname2@gmail.com', 2))
print("Number of rows updated: {}".format(cur.rowcount))

Number of rows updated: 1


In [ ]:
# access to metadata (tables and columns) using PRAGMA
cur.execute("PRAGMA table_info(SqliteDb_developers);")
data = cur.fetchall()
# print(data)
for d in data:
  print(d[0], d[1], d[2])

0 id INTEGER
1 name TEXT
2 email text
3 joining_date datetime
4 salary REAL


In [ ]:
# Excute whole SQL code in one step
cur.executescript("""
        DROP TABLE IF EXISTS patient_table;
        CREATE TABLE patient_table(patient_id INT, name TEXT, diagnose Text, birthdate datetime);
        INSERT INTO patient_table VALUES(10, 'name1', 'Asthma', '2006-02-28');
        INSERT INTO patient_table VALUES(20, 'name1', 'Asthma', '2007-02-28');
        INSERT INTO patient_table VALUES(30, 'name1', 'Asthma', '2008-02-28');
        INSERT INTO patient_table VALUES(40, 'name1', 'Asthma', '2009-02-28');
        DELETE FROM patient_table WHERE patient_id > 30;
     """)
conn.commit()
# cur.execute("DELETE FROM SqliteDb_developers WHERE id < 3")
cur.execute("SELECT COUNT(patient_id) FROM patient_table")
cur.fetchone()


(3,)

There are more information in:

http://zetcode.com/db/sqlitepythontutorial/

https://pynative.com/python-sqlite/

https://stackoverflow.com/questions/228912/sqlite-parameter-substitution-problem

# Exercises

### Connect to the database and see how many tables it contains




In [8]:
# Connect to source database
def create_connection(db_file):
  """ create a database connection to SQLite database"""
  conn = None
  try:
    conn = sqlite3.connect(db_file)
    return conn
  except Error as error:
    print("Error while connecting to sqlite", error)
  # finally:
  #   if conn:
  #     conn.close()

In [9]:
src_db = f"/content/drive/MyDrive/source_allergy_test.db"
src_conn = create_connection(src_db)
src_cur = src_conn.cursor()
print("Database connection is successfully")

Database connection is successfully


### Query the tables, join two tables



In [10]:

src_cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(src_cur.fetchall()) #die Namen aller Tabellen in der Datenmbank

[('disease',), ('patients',), ('encounters',), ('careplans',), ('conditions',), ('medications',), ('procedures',), ('observations',), ('devices',), ('imaging_studies',), ('immunizations',)]


In [11]:
src_cur.execute("SELECt * FROM patients")
records = src_cur.fetchall()
for row in records:
  print(row)

('27b0d72c-f2fb-7e25-38c0-7d5120ebbedf', '1945-04-17', '', '999-93-9618', 'S99971149', 'X1878420X', 'Mrs.', 'Emmie273', 'Schoen8', '', 'Heidenreich818', 'M', 'white', 'nonhispanic', 'F', 'Lexington  Massachusetts  US', '1056 Hegmann Mall', 'Kingston', 'Massachusetts', 'Plymouth County', 2364, 41.99610576466808, -70.78620468702952, 63141.11, 3176.7799999999997)
('17f0c6d9-8931-8839-66cb-3ca6fb066d3e', '1959-05-31', '', '999-17-6461', 'S99999085', 'X20851735X', 'Mr.', 'Karl184', 'Stroman228', '', '', 'M', 'white', 'nonhispanic', 'M', 'Sturbridge  Massachusetts  US', '1080 Boyer Brook Suite 22', 'Sutton', 'Massachusetts', 'Worcester County', '', 42.12612989310728, -71.7498484885064, 1333971.83, 4339.049999999999)
('aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5', '1991-11-27', '', '999-29-1688', 'S99979872', 'X48771954X', 'Mr.', 'Monty345', 'Krajcik437', '', '', 'M', 'white', 'nonhispanic', 'M', 'Watertown  Massachusetts  US', '657 Fay Road Suite 26', 'Haverhill', 'Massachusetts', 'Essex County', 1

In [2]:
import pandas as pd

#patient_pseudo = pd.read_sql_query("select id PATIENT, 'P'||random() ID from patients;", src_conn)
#patient_pseudo.head(10)

patient = pd.read_sql_query("select * from patients;", src_conn)
patient

medication = pd.read_sql_query("select * from medications;", src_conn)
medication



NameError: ignored

In [12]:
import pandas as pd
#patientmedication = pd.read_sql_query("select p.id, m.* from medications m join patients p on p.id = m.PATIENT", src_conn)
#patientmedication 
patientmedication = pd.read_sql_query("select distinct p.PATIENT, m.* from medications m join observations p on p.PATIENT = m.PATIENT",src_conn)
patientmedication 
#medication = pd.read_sql_query("select * from observations;", src_conn)
#medication

,PATIENT,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,1946-01-11T11:40:19Z,,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,6e0279b7-c477-a2c6-e75c-a911f03a264c,1049630,diphenhydrAMINE Hydrochloride 25 MG Oral Tablet,8.80,0.0,911,8016.80,,
1,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,2010-01-20T00:48:18Z,2010-10-20T00:48:18Z,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,047f6ec3-6215-35eb-9608-f9dda363a44c,be256df1-75cb-cbf7-0109-e0e1ae2c6348,314076,lisinopril 10 MG Oral Tablet,0.02,0.0,273,5.46,59621000,Hypertension
2,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,2010-02-19T00:48:18Z,2010-10-20T00:48:18Z,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,047f6ec3-6215-35eb-9608-f9dda363a44c,78f3c03e-8264-77b9-dc70-16fd00193b8d,310798,Hydrochlorothiazide 25 MG Oral Tablet,0.01,0.0,8,0.08,59621000,Hypertension
3,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,2010-04-20T00:48:18Z,2010-10-20T00:48:18Z,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,047f6ec3-6215-35eb-9608-f9dda363a44c,6f149158-46da-7ad4-040d-fd9786e66508,308136,amLODIPine 2.5 MG Oral Tablet,0.02,0.0,183,3.66,59621000,Hypertension
4,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,2010-10-20T00:48:18Z,2010-12-15T00:48:18Z,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,583a6d5e-4bda-34f7-06a5-97051581836f,310798,Hydrochlorothiazide 25 MG Oral Tablet,0.01,0.0,1,0.01,59621000,Hypertension
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8531,31698580-0ef9-5510-bea6-0c7ae94b0efc,2019-08-10T06:01:53Z,2020-08-15T06:01:53Z,31698580-0ef9-5510-bea6-0c7ae94b0efc,7caa7254-5050-3b5e-9eae-bd5ea30e809c,e2d9b3e3-f813-d8cd-8c5e-2af37e32a208,310798,Hydrochlorothiazide 25 MG Oral Tablet,0.01,0.0,12,0.12,59621000,Hypertension
8532,31698580-0ef9-5510-bea6-0c7ae94b0efc,2019-08-10T06:01:53Z,2020-08-15T06:01:53Z,31698580-0ef9-5510-bea6-0c7ae94b0efc,7caa7254-5050-3b5e-9eae-bd5ea30e809c,e2d9b3e3-f813-d8cd-8c5e-2af37e32a208,314076,lisinopril 10 MG Oral Tablet,0.02,0.0,371,7.42,59621000,Hypertension
8533,31698580-0ef9-5510-bea6-0c7ae94b0efc,2020-04-14T06:01:53Z,,31698580-0ef9-5510-bea6-0c7ae94b0efc,7caa7254-5050-3b5e-9eae-bd5ea30e809c,ab302aad-54f8-d9c8-1f34-3f156a65dd5e,314231,Simvastatin 10 MG Oral Tablet,0.02,0.0,12,0.24,55822004,Hyperlipidemia
8534,31698580-0ef9-5510-bea6-0c7ae94b0efc,2020-08-15T06:01:53Z,,31698580-0ef9-5510-bea6-0c7ae94b0efc,7caa7254-5050-3b5e-9eae-bd5ea30e809c,234e6b4c-ffcb-ac84-2070-3c77653dd4f8,310798,Hydrochlorothiazide 25 MG Oral Tablet,0.01,0.0,2,0.02,59621000,Hypertension


### Copy a table from one database to another(create empty SQLite database)

### Transform ETL Job: Joint two table and write them into a new table

### Write ETL jobs for all Tables




### Write a Program that schedules the ETL jobs and execute them in order
